In [ ]:
import torch
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import shutil

In [ ]:
UnityPath = '/home/data02/zjc/abaw/dataset/Unity'
for split in ['train', 'val']:
    for emo in ['Neutral', 'Happiness', 'Sadness', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt']:
        os.makedirs(os.path.join(f'dataset/Unity/{split}', emo), exist_ok=True)

# RAF-DB

In [ ]:
rafidx2emo = {1: 'Surprise', 2: 'Fear',3: 'Disgust',4: 'Happiness',5: 'Sadness', 6: 'Anger' , 7: 'Neutral'}
rafemo2idx = {emo: idx for idx, emo in rafidx2emo.items()}
print(rafidx2emo)
print(rafemo2idx)


In [ ]:
with open('dataset/RAFDB/basic/EmoLabel/list_patition_label.txt', 'r') as f:
    lines = f.readlines()

rafdb_name2idx = {}
for line in lines:
    name, idx = line.strip().split()
    name = name[:-4] + '_aligned.jpg'
    rafdb_name2idx[name] = int(idx)

# rafdb_name2idx

In [ ]:
rafdbpath = '/home/data02/zjc/abaw/dataset/RAFDB'


imgs = os.listdir(os.path.join(rafdbpath, 'basic/Image/aligned'))
for img in tqdm(imgs):
    # print(img, rafdb_name2idx[img])
    emotype = rafidx2emo[rafdb_name2idx[img]]

    img_org_path = os.path.join(rafdbpath, 'basic/Image/aligned', img)

    if img.startswith('train'):
        img_save_path = os.path.join(UnityPath, 'train', emotype, f'rafdb_{img}')
    elif img.startswith('test'):
        img_save_path = os.path.join(UnityPath, 'val', emotype, f'rafdb_{img}')
    # print(img_org_path)
    # print(img_save_path)
    
    shutil.copyfile(img_org_path, img_save_path)
    


# 复合表情

In [ ]:
rafdbpath = '/home/data02/zjc/abaw/dataset/RAFDB'
cepath = '/home/data02/zjc/abaw/dataset/RAFDBCE7'

ceidx2emo = {1: 'Happily Surprised', 2: 'Happily Disgusted',3: 'Sadly Fearful',4: 'Sadly Angry',
             5: 'Sadly Surprised',6: 'Sadly Disgusted',7: 'Fearfully Angry',8: 'Fearfully Surprised',
             9: 'Angrily Surprised',10: 'Angrily Disgusted',11: 'Disgustedly Surprised'}

ceemo2idx = {emo:idx for idx, emo in ceidx2emo.items()}
need = [8,1,5,11,9,3,4]
# need = range(1, 12)

name2idx = {}
for emo in ceidx2emo.values():
    if ceemo2idx[emo] in need:
        os.makedirs(os.path.join('dataset/RAFDBCE7', 'train', emo), exist_ok=True)
        os.makedirs(os.path.join('dataset/RAFDBCE7', 'val', emo), exist_ok=True)
    else:
        continue


with open('dataset/RAFDB/compound/EmoLabel/list_patition_label.txt', 'r') as f:
    lines = f.readlines()

for line in lines:
    name, idx = line.strip().split(' ')
    name2idx[name[:-4] + '_aligned.jpg'] = int(idx)

# print(name2idx)

imgs = os.listdir(os.path.join(rafdbpath, 'compound/Image/aligned'))


for img in tqdm(imgs):
    # print(img, name2idx[img])
    
    emotype = ceidx2emo[name2idx[img]]
    if name2idx[img] not in need:
        continue

    # break
    img_org_path = os.path.join(rafdbpath, 'compound/Image/aligned', img)

    if img.startswith('train'):
        img_save_path = os.path.join(cepath, 'train', emotype, f'rafdb_{img}')
    elif img.startswith('test'):
        img_save_path = os.path.join(cepath, 'val', emotype, f'rafdb_{img}')
    # print(img_org_path)
    # print(img_save_path)
    
    shutil.copyfile(img_org_path, img_save_path)
    # break


In [ ]:
import os
len(os.listdir('/home/data02/zjc/abaw/dataset/MAFW/data/clips'))

# MAFW

# AffectNet

In [ ]:
# labels = set()
exps = glob(os.path.join('dataset/AffectNet/val_set/annotations', '*_exp.npy'))
labels = set(np.load(i).item() for i in exps) # 0, ... ,7
affect_idx2emo = {0: 'Neutral', 1: 'Happiness', 2: 'Sadness', 3:'Surprise', 4: 'Fear', 5: 'Disgust', 6: 'Anger', 7: 'Contempt'}
affect_emo2idx = {emo: idx for idx, emo in affect_idx2emo.items()}

In [ ]:
affectnetpath = '/home/data02/zjc/abaw/dataset/AffectNet'
UnityPath = '/home/data02/zjc/abaw/dataset/Unity'



for split in os.listdir(affectnetpath):
    if split.endswith('.tar'):
        continue

    split_path = os.path.join(affectnetpath, split)
    split_path_imgs = os.path.join(split_path, 'images')

    for img in tqdm(os.listdir(split_path_imgs)):
        img_org_path = os.path.join(split_path_imgs, img)
        emo = int(np.load(os.path.join(split_path, 'annotations', img[:-4] + '_exp.npy')).item())
        emotype = affect_idx2emo[emo]
        img_save_path = os.path.join(UnityPath, split[:-4], emotype, f'affectnet_{img}')
        shutil.copyfile(img_org_path, img_save_path)


# MAFW

In [ ]:
mafw = '/home/data/zjc/code/MER2023-Baseline/abaw'
print(os.listdir(mafw))
np.load(os.path.join(mafw, 'models--facebook--wav2vec2-base-960h-FRA', '00019.npy')).shape
np.load(os.path.join(mafw, 'models--facebook--hubert-base-ls960-FRA', '00019.npy')).shape

In [ ]:
mpath = 'dataset/MAFW/Train & Test Set/compound/no_caption'
res = {}
for set_id in os.listdir(mpath):
    # print(set_id)
    with open(os.path.join(mpath, set_id, 'train.txt')) as f:
        lines = f.readlines()
    res[f'{set_id}_train'] = {line.strip().split()[0]: line.strip().split()[1] for line in lines}
    with open(os.path.join(mpath, set_id, 'test.txt')) as f:
        lines = f.readlines()
    res[f'{set_id}_test'] = {line.strip().split()[0]: line.strip().split()[1] for line in lines}

l = []
for set_id in res:
    l.append(len(res[set_id]))
print(l)

for i in range(0, 10, 2):
    print(l[i] + l[i+1])

mafw = 'dataset/MAFW'
frames = os.path.join(mafw, 'data', 'frames/frames')
len(os.listdir(frames)), len(os.listdir('/home/data/zjc/code/MER2023-Baseline/abaw/audio'))

In [ ]:
set1 = {}
set1.update(res['set_1_train'])
set1.update(res['set_1_test'])


In [ ]:
needed = ['fear_surprise', 'fear_surprise_anxiety', 
          'happiness_surprise',
            'sadness_surprise', 
           'disgust_surprise',
            'anger_surprise', 
            'fear_sadness', 'fear_sadness_anxiety',
           'anger_sadness']

In [ ]:
for clip in list(set1.keys()):
    if set1[clip] not in needed:
        set1.pop(clip)

mafwanno = {k[:-4]:v  for k, v in set1.items()}

save = {'train':[], 'test':[]}

for clip, emo in set1.items():
    if clip in res['set_1_train']:
        if emo not in ['fear_surprise_anxiety', 'fear_sadness_anxiety']:
            save['train'].append({'clip':clip[:-4], 'emo':emo})
        else:
            save['train'].append({'clip':clip[:-4], 'emo':emo[:-8]})
        
    elif clip in res['set_1_test']:
        if emo not in ['fear_surprise_anxiety', 'fear_sadness_anxiety']:
            save['test'].append({'clip':clip[:-4], 'emo':emo})
        else:
            save['test'].append({'clip':clip[:-4], 'emo':emo[:-8]})
    else:
        raise KeyError

np.save('mafwanno.npy', save)

In [ ]:
ff = 'dataset/MAFW/data/frames/frames'

for k in set1.keys():
    org = os.path.join(ff, k[:-4])
    new = os.path.join('dataset/MAFW_need', k[:-4])
    shutil.copytree(org, new)

In [ ]:
len(np.load('mafwanno.npy', allow_pickle=True).item()['train'])

In [ ]:
from collections import defaultdict

emoclip = defaultdict(list)
for k, v in set1.items():
    emoclip[v].append(k)
emoclip['fear_surprise'].extend(emoclip['fear_surprise_anxiety'])
emoclip.pop('fear_surprise_anxiety')
emoclip['fear_sadness'].extend(emoclip['fear_sadness_anxiety'])
emoclip.pop('fear_sadness_anxiety')

for k, v in emoclip.items():
    print(k, len(v))

In [ ]:
mafwce = '/home/data02/zjc/abaw/dataset/MAFWCE'
frame_path = '/home/data02/zjc/abaw/dataset/MAFW/data/frames/frames'

for emo, clips in emoclip.items():

    os.makedirs(os.path.join(mafwce, emo), exist_ok=True)
    for clip in clips:
        org_path = os.path.join(frame_path, clip[:-4])
        tgt_path = os.path.join(mafwce, emo, clip[:-4])
        shutil.copytree(org_path, tgt_path)

In [ ]:
import torch

In [ ]:
a = torch.load('manet/[02-08]-[21-19]-model_best-acc88.33.pth')

In [ ]:
np.load('mafwanno.npy')

In [ ]:
import matplotlib.pyplot as plt
with open('output/2024-03-16-004208/train.log', 'r') as f:
    lines = f.readlines()
lines = [float(line.strip().split(' ')[-1]) for line in lines if 'loss ' in line]
plt.plot(lines)

In [ ]:
import torch
checkpoint = torch.load('resnetferplus/best_bak_unity.pth')
pre_trained_dict = {k.replace('module.', ''): v for k,v in checkpoint.items()}

pre_trained_dict.pop('classifier.weight')
pre_trained_dict.pop('classifier.bias')
torch.save(pre_trained_dict, 'resnetferplus/ferplus/resnet50_ferplus_dag.pth')

In [ ]:
ls = []

for i in (glob('dataset/MAFWCE7/*/*')):
    ls.append(len(os.listdir(i)))

from collections import Counter
Counter(ls)

In [16]:
from sklearn.metrics import confusion_matrix, f1_score
import torch
import numpy as np
true, pred = torch.load('rafdb_ense_pred.pth')

In [17]:
f1_score(true, pred, average='macro')

0.7460161465490849

In [18]:
cm = confusion_matrix(true, pred)
print(cm)
print(cm.sum())



for i in range(len(cm)):
    print(cm[i][i] * 100 / cm.sum(axis=-1)[i])



[[ 23   0   5   4   6   0   0]
 [  4  22   5   1   0   0   3]
 [  2   2  98  12   1   1   0]
 [  3   2   4 123   1   1   1]
 [  1   1   0   0  27   4   0]
 [  0   0   2   1   0  17   2]
 [  0   1   2   2   0   2  11]]
397
60.526315789473685
62.857142857142854
84.48275862068965
91.11111111111111
81.81818181818181
77.27272727272727
61.111111111111114


In [12]:
import numpy as np
import torch
a, b, c,d = torch.load('predication/pred.pth',)

In [23]:
torch.load('mae/output_dir_base/checkpoint-0.pth')['model']['head.bias'].shape

torch.Size([1000])